# Bubble and Quick sort

## Common

In [ ]:
import random
import operator
import timeit


idx = operator.itemgetter(0)
val = operator.itemgetter(1)


def pairs(arr):
    return zip(enumerate(arr), enumerate(arr[1:], 1))


def is_bad(pair):
    a, b = pair
    return val(a) > val(b)


def bad_pairs(arr):
    return filter(is_bad, pairs(arr))


def is_sorted(arr):
    return not any(bad_pairs(arr))


def move(arr, at, to):
    arr.insert(to, arr.pop(at))
    

def test_array():
    return [random.randint(0, 100) for _ in range(25)]


def time(sorting_function):
    return timeit.timeit(f'{sorting_function.__name__}(test_array())', number=2000, globals=globals())


def print_sorting(sorting_function):
    print(f'{sorting_function.__name__} ({time(sorting_function)} sec)')
    a = test_array()
    print(a)
    assert not is_sorted(a)
    sorting_function(a)
    assert is_sorted(a)
    print(a)

## Bubble sort

In [ ]:
def bubble_sort(arr):
    while not is_sorted(arr):
        for a, b in bad_pairs(arr):
            move(arr, idx(a), idx(b))

## Quick sort

In [ ]:
def fo(arr, i, p):
    if arr[i] > arr[p]:
        move(arr, i, p)
        p -= 1
    else:
        i += 1
    return i, p


def foo(arr, i, p):
    while i < p:
        i, p = fo(arr, i, p)
    return p


def quick_sort(arr, start=0, end=-1):
    current_idx = start
    pivot_idx = end if end >= 0 else end + len(arr)
    if current_idx < pivot_idx:
        pivot_idx = foo(arr, current_idx, pivot_idx)
        quick_sort(arr, start, pivot_idx - 1)
        quick_sort(arr, pivot_idx + 1, end)

## Testing

In [ ]:
print_sorting(bubble_sort)
print()
print_sorting(quick_sort)